# Demo


This shows using Python to read data from the GitHub search
 API and writing it to the blob storage backed mount point



In [3]:
import json
import requests

req = requests.get("https://api.github.com/search/repositories?q=tetris+language:assembly&sort=stars&order=desc")
if not req.ok:
    raise "Unable to download data"

with open("/media/polydata/data/github-tetris.json", "w") as f:
    json.dump(req.json(), f)